In [ ]:
# Importing required libraries

import time
import requests
import re
from multiprocessing import Pool, cpu_count   

In [ ]:
# Downloading the text file from GitHub
file_path = 'https://raw.githubusercontent.com/MonaTlili/Assignment-MAS/refs/heads/main/Moby_dick/pg2701.txt'
response = requests.get(file_path, timeout=10) # Timeout is counted in seconds
sample_text = response.text

# Splitting the text into sentences for smaller chunks of data
text_cleaned = sample_text.replace('!', '.').replace('?', '.')
text_chunks = text_cleaned.split('.')

In [ ]:
# Map function to process a chunk of text, clean the text (by removing punctuations/underscores and converting to lowercase) and count word occurrences
def map_function(chunk):
    word_counts = {}
    cleaned_chunk = re.sub(r'[^\w\s]', '', chunk.lower())  
    cleaned_chunk = re.sub(r'_', '', cleaned_chunk)     
    words = cleaned_chunk.split()

    for word in words:
        word_counts[word] = word_counts.get(word, 0) + 1

    return word_counts

In [ ]:
# Combine function to merge word counts from multiple map results
def combine_results(results):
    combined_counts = {}

    for result in results:
        for word, count in result.items():
            combined_counts[word] = combined_counts.get(word, 0) + count

    return combined_counts

In [ ]:
# Parallel MapReduce implementation
def parallel_mapreduce(text_chunks):
    
    # Create a multiprocessing pool with the number of available CPU cores
    with Pool(cpu_count()) as pool:
        
        # Step 1: Map phase - distribute chunks to multiple processes
        map_results = pool.map(map_function, text_chunks)

        # Step 2: Shuffle and Reduce phase - aggregate results
        final_word_counts = combine_results(map_results)

    return final_word_counts

In [ ]:
    # Multi-threaded (MapReduce) execution with performance counter instead of time
    start_time = time.perf_counter()
    parallel_counts = parallel_mapreduce(chunks)
    parallel_duration = time.perf_counter() - start_time

    print("Parallel (MapReduce) Word Count:")
    print(parallel_counts)
    print(f"Parallel Duration: {parallel_duration:.6f} seconds\n")

    # Performance Summary
    print("Performance Comparison:")
    print(f"Single-threaded Time: {single_threaded_duration:.6f} seconds")
    print(f"Parallel Time: {parallel_duration:.6f} seconds")
    print(f"Speedup: {single_threaded_duration / parallel_duration:.2f}x (depending on dataset size)")